---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re



# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [9]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [48]:
def readGDPData():

    gdplev = pd.read_excel("../course1_downloads/gdplev.xls", header=0, names=["Quarter","GDP in billions of current dollars", "GDP in billions of chained 2009 dollars"], usecols="E:G", skiprows=7)

    gdp_data = pd.DataFrame(data=gdplev.iloc[-(2 + 16 * 4):])

    gdp_data = gdp_data.reset_index(drop=True)

    gdp_data = gdp_data.set_index(gdp_data.index)

    return gdp_data


def readHousingData():

    arr_slice = slice(0, 6)         # [0,6]; 0:6 is [0,6)
    us_home_prices = pd.read_csv(
        "../course1_downloads/City_Zhvi_AllHomes.csv", sep="\n", header=0, delimiter=",")

    us_home_prices = pd.DataFrame(
        data=us_home_prices, columns=us_home_prices.columns[arr_slice].append(us_home_prices.columns[-(12 * 16 + 8):]))

    us_home_prices.iloc[:, -(12 * 16 + 8):].apply(pd.to_numeric, axis=1)

    # Substitute the names of "State" for their full names
    for idx, stname in enumerate(us_home_prices["State"]):
        full_stname = states.get(stname)       
        us_home_prices.at[idx, "State"] = full_stname


    return us_home_prices

In [37]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # Read in the data first, then separate the data by state names
    state_pattern = re.compile(pattern="^.*\[edit\]$")
    states_and_regions = []
    with open("../course1_downloads/university_towns.txt", 'r', encoding='utf-8') as uni_towns:
        uni_towns.seek(0)
        curr_state = ""

        # Read the data line by line, deal with state and region names separately
        for line in uni_towns:
            if(re.match(state_pattern,line)):
                curr_state = line[0:line.find('[')]
            elif(line.find("(")): 
                # region = line[0:line.find('(') - 1].rstrip('\s') # Remove any extra whitespace chars on the right-hand side
                region = line[0:line.find('(') - 1]
                state_and_region = []
                state_and_region.append(curr_state)
                state_and_region.append(region)
                states_and_regions.append(state_and_region)
        
    # Organize the data in "[[State, RegionName], ...]" format
    # ans = pd.DataFrame(data=states_and_regions, columns=["State", "RegionName"], dtype=str).groupby(['State']).agg('size')
    ans = pd.DataFrame(data=states_and_regions, columns=["State", "RegionName"], dtype=str)
    # index = ans.columns
    # ans = ans.reset_index().set_index(index)
    # ans.index = ans["State"]
    # print(ans.head())
    # print("# of university towns: {}".format(len(ans)))
    return ans

# df = get_list_of_university_towns()
# print(df.where(df['State'] == "New Hampshire").count())

In [12]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    ### A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth. 
    
    gdp_data = readGDPData()

    # print(gdp_data.head())

    ans = ""

    for i in range(len(gdp_data) - 2):
        if(float(gdp_data.loc[i].loc['GDP in billions of chained 2009 dollars']) - float(gdp_data.loc[i + 1].loc['GDP in billions of chained 2009 dollars']) > 0 and float(gdp_data.loc[i + 1].loc['GDP in billions of chained 2009 dollars']) - float(gdp_data.loc[i + 2].loc['GDP in billions of chained 2009 dollars']) > 0): # Use the data under column 'GDP in billions of chained 2009 dollars' as it only takes into account the inflation from 2009 onwards
            ans = gdp_data.loc[i].Quarter
            break

    # print(gdp_data.loc[gdp_data['Quarter'] == '2008q3'])
    # print(gdp_data.loc[gdp_data['Quarter'] == '2008q4'])
    # print(gdp_data.loc[gdp_data['Quarter'] == '2009q1'])
    # return "ANSWER"
    return ans

get_recession_start()

'2008q2'

In [13]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    # A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

    gdp_data = readGDPData()

    recsn_start = get_recession_start() # How to determine the recession start? Determine by column "GDP in billions of current dollars" or "GDP in billions of chained 2009 dollars"?

    rescn_start_idx = gdp_data[gdp_data['Quarter'] == recsn_start].index.values[0]

    # print(rescn_start_idx.values)

    gdp_data_after_recsn_began = gdp_data.iloc[rescn_start_idx + 2:] # Here the starting index is 2 because the shortest duration of a recession is 3 quarters (2 consecutive quarters) 

    gdp_data_after_recsn_began = gdp_data_after_recsn_began.reset_index(drop=True)

    # print(gdp_data_after_recsn_began.head())

    ans = "ANSWER"

    for i in range(len(gdp_data_after_recsn_began) - 2): 
        if(float(gdp_data_after_recsn_began.loc[i].loc['GDP in billions of chained 2009 dollars']) - float(gdp_data_after_recsn_began.loc[i + 1].loc['GDP in billions of chained 2009 dollars']) < 0 and float(gdp_data_after_recsn_began.loc[i + 1].loc['GDP in billions of chained 2009 dollars']) - float(gdp_data_after_recsn_began.loc[i + 2].loc['GDP in billions of chained 2009 dollars']) < 0):
            ans =  gdp_data_after_recsn_began.loc[i + 2].Quarter
            break
    return ans

get_recession_end()

'2009q4'

In [14]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    # A recession bottom is the quarter within a recession which had the lowest GDP.
    gdp_data = readGDPData()

    recsn_start = get_recession_start()

    recsn_end = get_recession_end()

    recsn_start_idx = gdp_data[gdp_data.Quarter == recsn_start].index.values[0]
    # recsn_start_idx = gdp_data[gdp_data.Quarter == recsn_start].index

    # print(recsn_start_idx)

    recsn_end_idx = gdp_data[gdp_data.Quarter == recsn_end].index.values[0]
    # recsn_end_idx = gdp_data[gdp_data.Quarter == recsn_end].index
    
    # print(recsn_end_idx)
    
    gdp_data_in_recsn = gdp_data[recsn_start_idx:recsn_end_idx + 1]

    # print(gdp_data_in_recsn)    

    ans = "ANSWER"
    
    bottom_val = np.min(gdp_data_in_recsn['GDP in billions of current dollars'])

    # print(bottom_val)

    ans = gdp_data_in_recsn[gdp_data_in_recsn['GDP in billions of current dollars'] == bottom_val].loc[:,'Quarter'] # Series

    ans = ans.iloc[0] # String

    return ans

get_recession_bottom()

'2009q2'

In [47]:
def quarterMapper(date_str):
    if not type(date_str) == str:
        return date_str
    
    if not date_str.startswith("20"):
        return date_str

    mappings = {
        ("01","02","03"): "q1",
        ("04","05","06"): "q2",
        ("07","08","09"): "q3",
        ("10","11","12"): "q4",
    }
    year = date_str[0:4]
    month = date_str[-2:]
    ans = ""
    for key_strs in mappings.keys():
        quarter = ""
        for key_str in key_strs:
            if key_str == month:
                quarter = mappings.get(key_strs)
                break
        if not quarter == "":
            # ans = "".join([year,quarter])
            ans = pd.Period("".join([year,quarter]))
            break
            
    return ans

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = readHousingData()

    # Map the months to quarters

    # housing_data = (housing_data.groupby("State")
    #                             .agg({"State":"size"})
    # )

    # housing_data = housing_data.reset_index().iloc[:, 7:len(housing_data.columns)]

    housing_data = housing_data.reset_index()

    # housing_data = housing_data.rename(quarterMapper, axis='columns')

    housing_data = housing_data.rename(quarterMapper, axis='columns')

    # housing_data.set_index(["State", "RegionName"], inplace=True)
    # print(housing_data.head())

    # Need to change the data of housing prices to numeric data first
    # housing_data_by_quarters = housing_data.groupby(level=0, axis=1).agg('mean')
    housing_data_by_quarters = housing_data.iloc[:, -(12 * 16 + 8):].groupby(level=0, axis=1).agg('mean')

    # housing_data_by_quarters.apply(lambda x:x, axis=1)

    # print(housing_data_by_quarters.head())
    housing_data_by_quarters['State'] = housing_data['State']
    housing_data_by_quarters['RegionName'] = housing_data['RegionName']


    # index = pd.MultiIndex.from_frame()
    housing_data_by_quarters = housing_data_by_quarters.reset_index(drop=True).set_index(['State', 'RegionName'])

    # ans = pd.DataFrame(data=housing_data_by_quarters, index=['State', 'RegionName'])

    # print(housing_data_by_quarters)

    ans = housing_data_by_quarters
    # print(ans.head())

    return ans

# convert_housing_data_to_quarters().xs(('Alabama','Auburn'))

# quarterMapper("2010-01")
# quarterMapper("2010-04")
# quarterMapper("2010-08")
# quarterMapper("2010-11")

In [51]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_data = convert_housing_data_to_quarters()

    uni_towns = get_list_of_university_towns()

    # print("uni towns in California: {}".format(uni_towns[uni_towns.State == 'California']))

    st_and_reg_in_uni_towns = [uni_towns['State'], uni_towns['RegionName']]

    tuples = list(zip(*st_and_reg_in_uni_towns))
    # housing_data_of_uni_towns = housing_data.query("@housing_data['RegionName'] == @uni_towns")
    
    # print(uni_towns.head())

    # print(housing_data.xs("RegionName").head())

    # quarters_from_recession_start_to_bottom = [get_recession_start(), '2008q3', '2008q4', '2009q1', get_recession_bottom()]

    start = pd.Period(get_recession_start())

    bottom = pd.Period(get_recession_bottom())

    quarters_from_recession_start_to_bottom = [start,bottom]

    # print(repr(quarters_from_recession_start_to_bottom))
    # housing_data_of_uni_towns = housing_data[housing_data['RegionName'] == uni_towns]

    # print(housing_data_of_uni_towns.head())

    ### Get the housing prices btn the recession start and bottom
    housing_data_during_recession = housing_data.loc[:, quarters_from_recession_start_to_bottom]
    
    # housing_data_during_recession['ratio'] = housing_data.loc[:, start - 1] / housing_data.loc[:, bottom]

    housing_data_during_recession['ratio'] = housing_data.loc[:, start - 1] / housing_data.loc[:, bottom]

    # print(housing_data_during_recession['ratio'])
    # print(housing_data_during_recession.head())
    ### Acquire the data for uni-towns during that period
    
    uni_towns_data = None

    # uni_towns_data = pd.merge(uni_towns, housing_data_during_recession, how='inner', left_index=True, right_index=True)

    # print(len(uni_towns_data)) # 1059

    # uni_towns_data = None

    non_uni_towns_data = housing_data_during_recession

    for tup in tuples:
        try:
            # print(housing_data_during_recession.xs(tup))
            tup_data = housing_data_during_recession.xs(tup)
        except:
            # Ignore errors here
            continue
        tup_data = housing_data_during_recession.xs(tup)
        if(type(uni_towns_data) == type(None)):
            uni_towns_data = pd.DataFrame(data=tup_data)
        else:
            uni_towns_data = uni_towns_data.append(tup_data)
        non_uni_towns_data = non_uni_towns_data.drop(tup_data.index.values)
    # print(len(uni_towns_data)) # 269
    # print(len(non_uni_towns_data)) # 10461
    ### Acquire the data for non-uni towns during that period
    # print(uni_towns_data.head(10))
    # print(non_uni_towns_data.xs(('Alabama', 'Montevallo')))

    # Run the t-test and get the p-value
    # print(uni_towns_data.reset_index()[get_recession_start()])
    # print(non_uni_towns_data.reset_index()[get_recession_start()])
    # price_diff_uni_towns = uni_towns_data.loc[:, start] - uni_towns_data.loc[:, bottom]
    # price_diff_non_uni_towns = non_uni_towns_data.loc[:, start] - non_uni_towns_data.loc[:, bottom]
    # print(price_diff_uni_towns.head())
    # print(price_diff_non_uni_towns.head())
    # _,p = ttest_ind(price_diff_uni_towns, price_diff_non_uni_towns, nan_policy='omit')
    _,p = ttest_ind(uni_towns_data.ratio, non_uni_towns_data.ratio, nan_policy='omit')
    # print(p)
    # # Compute the mean price ratio and determine which is the better
    difference = (True if p < 0.01 else False)
    mean_ratio_uni_towns = uni_towns_data.ratio.mean()
    mean_ratio_non_uni_towns = non_uni_towns_data.ratio.mean()
    better = ("university town" if mean_ratio_uni_towns - mean_ratio_non_uni_towns < 0 else "non-university town")
    return (difference, p, better)
    # return "ANSWER"

run_ttest()